# Extract Additional Features from Cleaned Dataset

**Goal:** Extract unused features from `cleaned_data.pkl` and test if they improve F1 beyond 62.54%

**New features to extract:**
1. Topic Cluster Prominence Percentile
2. Publication link to Topic strength
3. Topic Cluster name (one-hot)
4. Language (is_english)
5. Pages (paper length)
6. Reference (number of references cited)
7. ASJC field name (subject area)
8. Sustainable Development Goals
9. Publisher

**These will be ADDED to existing 5,027 features, not replace them**

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## 1. Load Existing Features and Data

In [ ]:
# Load existing features (5,027 features)
X_train = pd.read_pickle('../data/features/X_train_temporal.pkl')
X_test = pd.read_pickle('../data/features/X_test_temporal.pkl')
y_train = pd.read_pickle('../data/features/y_train_cls_temporal.pkl')
y_test = pd.read_pickle('../data/features/y_test_cls_temporal.pkl')

# Load metadata to get indices
metadata_train = pd.read_pickle('../data/features/metadata_train.pkl')
metadata_test = pd.read_pickle('../data/features/metadata_test.pkl')

# Load full cleaned dataset
df = pd.read_pickle('../data/processed/cleaned_data.pkl')

print(f"Existing features - Train: {X_train.shape}")
print(f"Existing features - Test: {X_test.shape}")
print(f"Full dataset: {df.shape}")
print(f"\nCurrent feature count: {X_train.shape[1]}")

## 2. Extract New Features from Cleaned Dataset

In [ ]:
def extract_additional_features(df, indices):
    """
    Extract new features from cleaned dataset that weren't used before.
    Only use features observable at publication time (ex-ante).
    """
    # Subset to the indices we need
    df_subset = df.loc[indices].copy()
    
    new_features = pd.DataFrame(index=indices)
    
    # 1. Topic Cluster Prominence Percentile
    new_features['topic_cluster_prominence'] = pd.to_numeric(
        df_subset['Topic Cluster Prominence Percentile'], 
        errors='coerce'
    )
    
    # 2. Publication link to Topic strength
    new_features['topic_link_strength'] = pd.to_numeric(
        df_subset['Publication link to Topic strength'], 
        errors='coerce'
    )
    
    # 3. Language (is_english binary)
    new_features['is_english'] = (df_subset['Language'] == 'English').astype(int)
    
    # 4. Pages (paper length)
    # Extract numeric page count from Pages column (handles ranges like "1-10")
    def extract_page_count(pages_str):
        if pd.isna(pages_str) or pages_str == '':
            return np.nan
        # Handle ranges like "123-145" or single numbers "5"
        try:
            if '-' in str(pages_str):
                parts = str(pages_str).split('-')
                start = int(parts[0].strip())
                end = int(parts[1].strip())
                return end - start + 1
            else:
                return 1  # Single page
        except:
            return np.nan
    
    new_features['page_count'] = df_subset['Pages'].apply(extract_page_count)
    
    # 5. Reference count (number of references cited)
    new_features['reference_count'] = pd.to_numeric(
        df_subset['Reference'], 
        errors='coerce'
    )
    
    # 6. SDG binary (has Sustainable Development Goal)
    new_features['has_sdg'] = df_subset['Sustainable Development Goals (2025)'].notna().astype(int)
    
    # 7. Subject area (ASJC field name) - one-hot encode top 10 most common
    asjc_field = df_subset['All Science Journal Classification (ASJC) field name'].fillna('Unknown')
    # Get top 10 most common fields
    top_fields = asjc_field.value_counts().head(10).index.tolist()
    for field in top_fields:
        new_features[f'asjc_{field[:20]}'] = (asjc_field == field).astype(int)
    
    # 8. Publisher - one-hot encode top 5
    publisher = df_subset['Publisher'].fillna('Unknown')
    top_publishers = publisher.value_counts().head(5).index.tolist()
    for pub in top_publishers:
        new_features[f'publisher_{pub[:15]}'] = (publisher == pub).astype(int)
    
    # 9. Topic Cluster - one-hot encode top 10 clusters
    topic_cluster = df_subset['Topic Cluster name'].fillna('Unknown')
    top_clusters = topic_cluster.value_counts().head(10).index.tolist()
    for cluster in top_clusters:
        new_features[f'cluster_{cluster[:20]}'] = (topic_cluster == cluster).astype(int)
    
    # Impute missing numeric values with median
    numeric_cols = ['topic_cluster_prominence', 'topic_link_strength', 'page_count', 'reference_count']
    for col in numeric_cols:
        if col in new_features.columns:
            median_val = new_features[col].median()
            new_features[col].fillna(median_val, inplace=True)
    
    return new_features

# Extract features for train and test
print("Extracting new features...")
new_features_train = extract_additional_features(df, X_train.index)
new_features_test = extract_additional_features(df, X_test.index)

print(f"\nNew features extracted: {new_features_train.shape[1]}")
print(f"\nNew feature columns:")
for i, col in enumerate(new_features_train.columns, 1):
    print(f"  {i:2}. {col}")

## 3. Combine New Features with Existing Features

In [ ]:
# Combine existing 5,027 features with new features
X_train_expanded = pd.concat([X_train, new_features_train], axis=1)
X_test_expanded = pd.concat([X_test, new_features_test], axis=1)

print("=" * 60)
print("FEATURE EXPANSION SUMMARY")
print("=" * 60)
print(f"Original features: {X_train.shape[1]}")
print(f"New features added: {new_features_train.shape[1]}")
print(f"Total features now: {X_train_expanded.shape[1]}")
print(f"\nTrain shape: {X_train_expanded.shape}")
print(f"Test shape: {X_test_expanded.shape}")

## 4. Test Models with Expanded Features

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, threshold=0.54, name="Model"):
    """Train and evaluate a model"""
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    results = {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }
    
    return results, model

results_list = []

### Baseline (Original 5,027 features)

In [ ]:
print("=" * 60)
print("BASELINE - Original 5,027 features")
print("=" * 60)

baseline_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

baseline_results, _ = evaluate_model(baseline_model, X_train, y_train, X_test, y_test, name="Baseline (5,027 features)")

for metric, value in baseline_results.items():
    if metric != 'Model':
        print(f"{metric:12s}: {value*100:.2f}%")

results_list.append(baseline_results)

### Experiment 1: Logistic Regression with Expanded Features

In [ ]:
print("\n" + "=" * 60)
print(f"EXPERIMENT 1 - LR with Expanded Features ({X_train_expanded.shape[1]} features)")
print("=" * 60)

exp1_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

exp1_results, _ = evaluate_model(exp1_model, X_train_expanded, y_train, X_test_expanded, y_test, name=f"LR Expanded ({X_train_expanded.shape[1]} features)")

print("\nResults:")
for metric, value in exp1_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp1_results)

### Experiment 2: Random Forest with Expanded Features

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 2 - Random Forest with Expanded Features")
print("=" * 60)

exp2_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

print("Training Random Forest...")
exp2_results, _ = evaluate_model(exp2_model, X_train_expanded, y_train, X_test_expanded, y_test, name="RF Expanded")

print("\nResults:")
for metric, value in exp2_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp2_results)

### Experiment 3: LightGBM with Expanded Features

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 3 - LightGBM with Expanded Features")
print("=" * 60)

exp3_model = lgb.LGBMClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced',
    verbose=-1
)

print("Training LightGBM...")
exp3_results, _ = evaluate_model(exp3_model, X_train_expanded, y_train, X_test_expanded, y_test, name="LightGBM Expanded")

print("\nResults:")
for metric, value in exp3_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp3_results)

### Experiment 4: Stacking with Expanded Features

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 4 - Stacking Ensemble with Expanded Features")
print("=" * 60)

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

stacking_expanded = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', max_depth=10)),
        ('lgbm', lgb.LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced', verbose=-1)),
        ('xgb', xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=scale_pos_weight))
    ],
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5,
    n_jobs=-1
)

print("Training stacking ensemble (this will take several minutes)...")
exp4_results, _ = evaluate_model(stacking_expanded, X_train_expanded, y_train, X_test_expanded, y_test, name="Stacking Expanded")

print("\nResults:")
for metric, value in exp4_results.items():
    if metric != 'Model':
        baseline_val = baseline_results[metric]
        diff = (value - baseline_val) * 100
        print(f"{metric:12s}: {value*100:.2f}% ({diff:+.2f})")

results_list.append(exp4_results)

## 5. Final Comparison

In [ ]:
print("\n" + "=" * 80)
print("FINAL COMPARISON - Expanded Features vs Baseline")
print("=" * 80)

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('F1', ascending=False)

# Format as percentages
display_df = results_df.copy()
for col in ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC-AUC']:
    display_df[col] = display_df[col].apply(lambda x: f"{x*100:.2f}%")

print("\n", display_df.to_string(index=False))

# Find best F1
print("\n" + "=" * 80)
best_model = results_df.iloc[0]

print(f"\n🏆 BEST MODEL: {best_model['Model']}")
print(f"   F1 Score: {best_model['F1']*100:.2f}%")
print(f"   Improvement over baseline: {(best_model['F1'] - baseline_results['F1'])*100:+.2f} points")
print(f"   ROC-AUC: {best_model['ROC-AUC']*100:.2f}%")
print(f"   Precision: {best_model['Precision']*100:.2f}%")
print(f"   Recall: {best_model['Recall']*100:.2f}%")
print("=" * 80)

## 6. Feature Importance Analysis (if improvement found)

In [ ]:
improvement = (best_model['F1'] - baseline_results['F1']) * 100

if improvement > 0.3:
    print("\n" + "=" * 60)
    print("FEATURE IMPORTANCE - Top 20 Most Important Features")
    print("=" * 60)
    
    # Train a simple model to get feature importances
    importance_model = lgb.LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced', verbose=-1)
    importance_model.fit(X_train_expanded, y_train)
    
    # Get feature importances
    feature_importance = pd.DataFrame({
        'feature': X_train_expanded.columns,
        'importance': importance_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 20 features:")
    print(feature_importance.head(20).to_string(index=False))
    
    # Check how many new features are in top 20
    new_feature_names = new_features_train.columns.tolist()
    top_20_new_features = feature_importance.head(20)['feature'].isin(new_feature_names).sum()
    
    print(f"\n✅ New features in top 20: {top_20_new_features} / 20")
    print(f"   This shows the new features are {'highly' if top_20_new_features > 5 else 'moderately' if top_20_new_features > 2 else 'minimally'} important")
else:
    print("\n⚠️  Improvement too small to analyze feature importance")

## 7. Recommendation

In [ ]:
print("\n💡 FINAL RECOMMENDATION:")
print("-" * 80)

improvement = (best_model['F1'] - baseline_results['F1']) * 100

if improvement > 1.0:
    print(f"✅ SIGNIFICANT IMPROVEMENT FOUND!")
    print(f"   Use '{best_model['Model']}' for final model")
    print(f"   New features added {new_features_train.shape[1]} features")
    print(f"   F1 improved from {baseline_results['F1']*100:.2f}% → {best_model['F1']*100:.2f}% (+{improvement:.2f} points)")
    print(f"\n   These additional features from cleaned_data.pkl were worth extracting!")
    print(f"   Total feature count now: {X_train_expanded.shape[1]}")
elif improvement > 0.3:
    print(f"⚠️  MODERATE IMPROVEMENT (+{improvement:.2f} F1)")
    print(f"   New features provide small but measurable gain")
    print(f"   F1: {baseline_results['F1']*100:.2f}% → {best_model['F1']*100:.2f}%")
    print(f"   Consider if {new_features_train.shape[1]} extra features worth the complexity")
else:
    print(f"❌ MINIMAL/NO IMPROVEMENT (+{improvement:.2f} F1)")
    print(f"   Additional features from cleaned_data.pkl did not help")
    print(f"   Stick with baseline {baseline_results['F1']*100:.2f}% F1")
    print(f"\n   The original 5,027 features already captured the predictive signal.")

print("\n" + "-" * 80)
print(f"Summary:")
print(f"  Baseline (5,027 features): {baseline_results['F1']*100:.2f}% F1")
print(f"  Best expanded model: {best_model['F1']*100:.2f}% F1")
print(f"  New features added: {new_features_train.shape[1]}")
print(f"  Total features: {X_train_expanded.shape[1]}")

## 8. Save Expanded Features (if improvement found)

In [ ]:
# Uncomment to save if improvement is significant

# if improvement > 0.5:
#     output_dir = Path('../data/features')
#     output_dir.mkdir(exist_ok=True)
#     
#     X_train_expanded.to_pickle(output_dir / 'X_train_expanded.pkl')
#     X_test_expanded.to_pickle(output_dir / 'X_test_expanded.pkl')
#     new_features_train.to_pickle(output_dir / 'extra_features_train.pkl')
#     new_features_test.to_pickle(output_dir / 'extra_features_test.pkl')
#     
#     print("✓ Expanded features saved!")
#     print(f"  X_train_expanded.pkl: {X_train_expanded.shape}")
#     print(f"  X_test_expanded.pkl: {X_test_expanded.shape}")
# else:
#     print("No significant improvement - not saving expanded features")